In [1]:
import pandas as pd
import numpy as np

ohlc_path = "/home/mimus/qlearning/data/sources/candles/binance/BINANCE_BTCUSDT.historic_prices_5m.json"
ew_path = "/home/mimus/qlearning/data/sources/ew/binance/BINANCE_BTCUSDT.strategies_5_t5001.json"

ohlc_df = pd.read_json(ohlc_path)
ohlc_df=ohlc_df[["ts","o","h","l", "c"]].iloc[0:-286]#hay que cambiar esto para que se usen los ts en lugar de estas cosas raras
ohlc_df=ohlc_df.reset_index(drop=True)
ew_df = pd.read_json(ew_path)
ew_df=ew_df.sort_values(by=['ts']).iloc[1:].reset_index(drop=True)

In [2]:
merged_df= pd.merge(ohlc_df, ew_df, on='ts', how='left')
merged_df.fillna(0, inplace=True)
merged_df.to_csv("data/training/5m_train_qlearning.csv")
